# Validation

> Team Name: *S Legends*
>
> Team Members: Myles, Tani, Arjan, Archie

In [106]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import statsmodels.api as sm
import statsmodels.tools
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

**The validation method we use is K-fold cross-validation:**  

**What is K-fold cross-validation?**  
This splits up the whole dataset into a K number of subsets (folds), and then uses K-1 folds as the training set and validates on the remaining 1 fold.

**Why use K-fold cross_validation?**  
This means that every observation in the data is used to test one time and it can provide a better estimate of the accuracy of the model.

In [19]:
df = pd.read_csv('Life Expectancy Data.csv') # Load the original dataset

In [189]:
kf = KFold(n_splits = 5, shuffle=True, random_state = 42) # Initialise the KFold function, informing it how many splits to make

In [164]:
feature_cols = list(df.columns)
feature_cols.remove('Life_expectancy')

X = df[feature_cols]
y = df['Life_expectancy']

In [193]:
fold_indices = [] # Store the train-test indices as a tuple for each fold

for i, (train_indices, test_indices) in enumerate(kf.split(X)):
    fold_indices.append((train_indices, test_indices))

Feature Engineer using function made previously:

In [205]:
# Function created to feature engineer.
def feature_engineering(X):
    
    X = X.drop(columns = ["Country", "Region", "Year"])
    
    # Remove highly correlated features to reduce multicollinearity.
    corr_matrix = X.corr().abs() # Creates a correlation matrix for all the values in the X dataframe, making sure its absolute values (this prevents a division by zero error).
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(bool)) # Takes the upper triangle of the correlation matrix, k = 1, is the upper diagonal, and changes the type to boolean.
    to_drop = [column for column in upper.columns if any(upper[column] > 0.9)] # Takes all the columns with correlation greater than 0.9 in one list.
    X = X.drop(columns=to_drop) # Drops all of the columns with the correlations > 0.9 - this is the feature selection step.

    # Remove features with low variance or high VIF
    X_scaled = StandardScaler().fit_transform(X) # Uses the StandardScaler() from sklearn and uses fit_transform on X to scale it using the mean and standard deviation.
    X_df = pd.DataFrame(X_scaled, columns = X.columns) # Converts the above X_scaled back into a dataframe - as it is an np array.
    
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns # Adds a column called feature and fills it with the columns from X.
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])] # Iterates through and calculates the VIF for each feature.
    X = X_df.drop(columns = vif_data[vif_data["VIF"] > 10]["feature"]) # Drops the columns with a VIF value > 10, as this suggests high multicollinearity.

    # Returns the feature engineered version of X.
    return X

**Validation:**

In [211]:
def validation(kf, fold_indices, X, y):
    rmse = []
    rsquared = []
    cond_no = []
    
    for fold in range(kf.get_n_splits(X)+1):

        # Training on the fold's training set
        train_index = fold_indices[fold][0]
        test_index = fold_indices[fold][1]
        
        X_fold_train = X.loc[train_index]
        y_fold_train = y.loc[train_index]
        
        X_fold_train_fe = feature_engineering(X_fold_train)
        y_fold_train = y_fold_train.loc[X_fold_train_fe.index] 

        model = sm.OLS(y_fold_train, X_fold_train_fe)
        results = model.fit()


        # Testing on the fold's testing set
        X_fold_test = X.loc[test_index]
        y_fold_test = y.loc[test_index]

        X_fold_test_fe = feature_engineering(X_fold_test)
        y_fold_test = y_fold_test.loc[X_fold_test_fe.index]
        y_pred = results.predict(X_fold_test_fe)
    
        R_squared1 = results.rsquared
        cond_no1 = results.condition_number
        rmse1 = statsmodels.tools.eval_measures.rmse(y_fold_test, y_pred)

        rmse.append(rmse1)
        rsquared.append(R_squared1)
        cond_no.append(cond_no1)

    mean_rmse = np.mean(rmse)
    mean_rsquared = np.mean(rsquared)
    mean_cond_no = np.mean(cond_no)

    return mean_rmse, mean_rsquared, mean_cond_no

In [213]:
(rmse, rsquared, mean_cond_no) =  validation(kf, fold_indices, X, y)

KeyError: '[29, 30, 32, 43, 44, 45, 51, 56, 63, 67, 70, 73, 80, 87, 93, 96, 102, 104, 108, 109, 111, 124, 134, 135, 141, 149, 152, 163, 168, 170, 173, 175, 178, 188, 194, 196, 199, 210, 211, 212, 218, 221, 226, 231, 233, 237, 239, 246, 251, 252, 254, 257, 259, 270, 272, 282, 289, 291, 296, 298, 309, 313, 314, 318, 321, 322, 324, 331, 332, 354, 358, 365, 367, 368, 381, 387, 393, 402, 407, 408, 410, 411, 414, 415, 420, 422, 430, 432, 433, 435, 436, 439, 443, 450, 456, 457, 460, 462, 463, 464, 471, 478, 479, 485, 486, 495, 498, 506, 507, 509, 511, 518, 521, 527, 528, 532, 533, 535, 542, 543, 544, 554, 557, 564, 565, 567, 568, 572, 581, 594, 598, 599, 602, 605, 610, 612, 642, 644, 650, 655, 662, 676, 677, 678, 679, 685, 691, 693, 695, 700, 705, 707, 727, 736, 741, 742, 744, 755, 759, 765, 767, 772, 783, 785, 787, 790, 794, 798, 802, 807, 809, 817, 819, 831, 834, 840, 841, 847, 857, 869, 879, 889, 900, 903, 907, 911, 912, 926, 927, 930, 937, 940, 941, 942, 958, 962, 965, 978, 986, 999, 1005

In [201]:
train_index = fold_indices[0][0]

In [203]:
train_index

array([   0,    1,    2, ..., 2861, 2862, 2863])